In [1]:
# --- Импорт библиотек ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_score, recall_score

# --- Настройки ---
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (15, 7)

# --- Загрузка ---
file_path = 'data/База.csv'
try:
    df_raw = pd.read_csv(file_path, sep=';')
    print(f"Данные успешно загружены. Форма: {df_raw.shape}")
except FileNotFoundError:
    print(f"!!! КРИТИЧЕСКАЯ ОШИБКА: Файл не найден по пути '{file_path}'.")

Данные успешно загружены. Форма: (5519, 21)


In [2]:
# --- Шаг 2: Фильтрация ---
df_filtered = df_raw[df_raw['ВидПомещения'].str.strip() == 'жилые помещения'].copy()
final_statuses = ['Продана', 'Свободна']
df_final = df_filtered[df_filtered['СледующийСтатус'].isin(final_statuses)].copy()

df_final['target'] = df_final['СледующийСтатус'].map({'Продана': 1, 'Свободна': 0})
df_final.drop(['УИД_Брони', 'ВидПомещения', 'СледующийСтатус'], axis=1, inplace=True)
print(f"Данные отфильтрованы. Размер для обработки: {df_final.shape}")

Данные отфильтрованы. Размер для обработки: (3944, 19)


In [4]:
# --- Шаги 3 и 4: Надежная Очистка, Преобразование и Заполнение пропусков (ФИНАЛЬНАЯ ВЕРСИЯ) ---

df_processed = df_final.copy()
print(f"\n--- Начало очистки. Размер: {df_processed.shape} ---")

# 3.1. Преобразуем все потенциально числовые столбцы, исправляя запятые
numeric_cols = [
    'ПродаваемаяПлощадь', 'Этаж', 'СтоимостьНаДатуБрони', 
    'СкидкаНаКвартиру', 'ФактическаяСтоимостьПомещения', 'Тип'
]
for col in numeric_cols:
    if df_processed[col].dtype == 'object':
        if col == 'Тип':
            df_processed[col] = df_processed[col].str.replace('к', '', regex=False)
        df_processed[col] = df_processed[col].str.replace(',', '.', regex=False)
    
    df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
print("Числовые столбцы преобразованы.")


# 3.2. Бинарное кодирование (НЕЧУВСТВИТЕЛЬНОЕ К РЕГИСТРУ И NaN)
binary_cols_map = {
    'ВременнаяБронь': {'да': 1, 'нет': 0},
    'СделкаАН': {'да': 1, 'нет': 0},
    'ИнвестиционныйПродукт': {'да': 1, 'нет': 0},
    'Привилегия': {'да': 1, 'нет': 0}
}
for col, mapping in binary_cols_map.items():
    # .str.lower() решает проблему регистра, а fillna('') перед map решает проблему NaN
    df_processed[col] = df_processed[col].str.lower().map(mapping)

# Отдельно обработаем более сложные бинарные признаки
df_processed['ИсточникБрони'] = df_processed['ИсточникБрони'].map({'ручная': 0, 'МП': 1})

# >>>>> ВОТ ИСПРАВЛЕНИЕ <<<<<
# Добавляем na=False, чтобы обработать NaN до конвертации в int
df_processed['ТипСтоимости'] = df_processed['ТипСтоимости'].str.contains('100%', na=False).astype(int)
df_processed['ВариантОплаты'] = df_processed['ВариантОплаты'].str.contains('Единовременная', na=False).astype(int)
print("Бинарные признаки закодированы.")


# 3.3. Категориальное кодирование
df_processed = pd.get_dummies(df_processed, columns=['Город', 'Статус лида (из CRM)'], drop_first=True, dtype=int)
print("Категориальные признаки закодированы.")


# 3.4. Удаляем ненужные столбцы
df_processed.drop(['ДатаБрони', 'ВремяБрони', 'ВариантОплатыДоп'], axis=1, inplace=True, errors='ignore')

# 4. УМНОЕ ЗАПОЛНЕНИЕ ПРОПУСКОВ
print("\n--- Заполнение пропусков ---")
for col in df_processed.columns:
    if df_processed[col].isnull().any():
        if pd.api.types.is_numeric_dtype(df_processed[col]):
            median_val = df_processed[col].median()
            df_processed[col].fillna(median_val, inplace=True)
            print(f"Числовой столбец '{col}': пропуски заменены медианой ({median_val:.2f})")
        else: # Этот блок теперь, скорее всего, не понадобится, но оставим для надежности
            mode_val = df_processed[col].mode()[0]
            df_processed[col].fillna(mode_val, inplace=True)
            print(f"Категориальный столбец '{col}': пропуски заменены модой ('{mode_val}')")

# ФИНАЛЬНАЯ ПРОВЕРКА
if df_processed.isnull().sum().sum() == 0:
    print(f"\nОчистка завершена успешно. Финальный размер: {df_processed.shape}")
else:
    raise RuntimeError("ОШИБКА: После очистки все еще остались пропуски!")


--- Начало очистки. Размер: (3944, 19) ---
Числовые столбцы преобразованы.
Бинарные признаки закодированы.
Категориальные признаки закодированы.

--- Заполнение пропусков ---
Числовой столбец 'Тип': пропуски заменены медианой (2.00)
Числовой столбец 'ПродаваемаяПлощадь': пропуски заменены медианой (60.50)
Числовой столбец 'СкидкаНаКвартиру': пропуски заменены медианой (111876.00)

Очистка завершена успешно. Финальный размер: (3944, 24)


C:\Users\Thero\AppData\Local\Temp\ipykernel_18544\3615649076.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].fillna(median_val, inplace=True)
C:\Users\Thero\AppData\Local\Temp\ipykernel_18544\3615649076.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

In [5]:
# --- Шаг 5: Feature Engineering ---
epsilon = 1e-6
df_processed['ЦенаЗаМетр'] = df_processed['ФактическаяСтоимостьПомещения'] / (df_processed['ПродаваемаяПлощадь'] + epsilon)
initial_price = df_processed['ФактическаяСтоимостьПомещения'] + df_processed['СкидкаНаКвартиру']
df_processed['СкидкаПроцент'] = (df_processed['СкидкаНаКвартиру'] / (initial_price + epsilon)) * 100
print("Созданы новые признаки.")

# --- Шаги 6-9: Подготовка к обучению ---
y = df_processed['target']
X = df_processed.drop('target', axis=1)
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_scaled['СкидкаНаКвартиру'] = (X_scaled['СкидкаНаКвартиру'] - 0.5)
print(f"Баланс классов: \n{y.value_counts(normalize=True)}")
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)
print(f"Данные готовы. Train: {X_train.shape}, Test: {X_test.shape}")


# --- Шаги 10-14: Построение и оценка моделей ---
def evaluate_model(model, model_name):
    print(f"\n--- Оценка модели: {model_name} ---")
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    precision = precision_score(y_test, y_pred_test)
    recall = recall_score(y_test, y_pred_test)
    f1 = f1_score(y_test, y_pred_test)
    print(f"  Precision: {precision:.4f} | Recall: {recall:.4f} | F1-мера: {f1:.4f}")
    return {'model': model_name, 'precision': precision, 'recall': recall, 'f1_score': f1}

# Обучение
knn = KNeighborsClassifier()
knn_results = evaluate_model(knn, "K-Nearest Neighbors (KNN)")

tree = DecisionTreeClassifier(random_state=42)
tree_results = evaluate_model(tree, "Decision Tree")

# Вывод
print("\n--- Итоговое сравнение моделей ---")
results_df = pd.DataFrame([knn_results, tree_results]).sort_values('f1_score', ascending=False)
display(results_df)

Созданы новые признаки.
Баланс классов: 
target
0    0.710953
1    0.289047
Name: proportion, dtype: float64
Данные готовы. Train: (2760, 25), Test: (1184, 25)

--- Оценка модели: K-Nearest Neighbors (KNN) ---
  Precision: 0.7491 | Recall: 0.6199 | F1-мера: 0.6784

--- Оценка модели: Decision Tree ---
  Precision: 0.8018 | Recall: 0.7690 | F1-мера: 0.7851

--- Итоговое сравнение моделей ---


,model,precision,recall,f1_score
1,Decision Tree,0.801829,0.769006,0.785075
0,K-Nearest Neighbors (KNN),0.749117,0.619883,0.678400
